In [2]:
import os
from dotenv import load_dotenv



In [45]:
load_dotenv()

True

In [46]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [47]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [7]:
!mkdir data

In [8]:
from langchain.document_loaders import PyPDFLoader

In [14]:
file_path = "data/ML.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [15]:
len(data)

118

In [16]:
question_gen = ""

for page in data:
    question_gen += page.page_content

In [17]:
question_gen

" \n \n \n \n \n \n \nMachine L earning Year ning is a   \ndeeplear ning. ai pr oject. \n \n \n \n \n \n \n \n \n \n \n© 2018 Andrew Ng.  All Rights R eserv ed. \n \n  \nPage 2 Machine Learning Yearning-Draft Andrew Ng Deeplearning.AI \nTable of Contents \n \n1 Why Machine Learning Strategy  \n2 How to use this book to help your team  \n3 Prerequisites and Notation  \n4 Scale drives machine learning progress  \n5 Your development and test sets  \n6 Your dev and test sets should come from the same distribution  \n7 How large do the dev/test sets need to be?  \n8 Establish a single-number evaluation metric for your team to optimize  \n9 Optimizing and satisficing metrics  \n10 Having a dev set and metric speeds up iterations  \n11 When to change dev/test sets and metrics  \n12 Takeaways: Setting up development and test sets  \n13 Build your first system quickly, then iterate  \n14 Error analysis: Look at dev set examples to evaluate ideas  \n15 Evaluating multiple ideas in parallel durin

In [18]:
from langchain.text_splitter import TokenTextSplitter

In [20]:
splitter_ques_gen = TokenTextSplitter(
    model_name = "gpt-3.5-turbo",
    chunk_size = 10000,
    chunk_overlap = 200
)

In [21]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [22]:
len(chunk_ques_gen)

4

In [23]:
chunk_ques_gen

[' \n \n \n \n \n \n \nMachine L earning Year ning is a   \ndeeplear ning. ai pr oject. \n \n \n \n \n \n \n \n \n \n \n© 2018 Andrew Ng.  All Rights R eserv ed. \n \n  \nPage 2 Machine Learning Yearning-Draft Andrew Ng Deeplearning.AI \nTable of Contents \n \n1 Why Machine Learning Strategy  \n2 How to use this book to help your team  \n3 Prerequisites and Notation  \n4 Scale drives machine learning progress  \n5 Your development and test sets  \n6 Your dev and test sets should come from the same distribution  \n7 How large do the dev/test sets need to be?  \n8 Establish a single-number evaluation metric for your team to optimize  \n9 Optimizing and satisficing metrics  \n10 Having a dev set and metric speeds up iterations  \n11 When to change dev/test sets and metrics  \n12 Takeaways: Setting up development and test sets  \n13 Build your first system quickly, then iterate  \n14 Error analysis: Look at dev set examples to evaluate ideas  \n15 Evaluating multiple ideas in parallel duri

In [24]:
type(chunk_ques_gen)

list

In [25]:
type(chunk_ques_gen[0])

str

In [26]:
# converting text to document
from langchain.docstore.document import Document

In [27]:
document_ques_gen = [Document(page_content=t) for t in chunk_ques_gen]

In [28]:
document_ques_gen

[Document(page_content=' \n \n \n \n \n \n \nMachine L earning Year ning is a   \ndeeplear ning. ai pr oject. \n \n \n \n \n \n \n \n \n \n \n© 2018 Andrew Ng.  All Rights R eserv ed. \n \n  \nPage 2 Machine Learning Yearning-Draft Andrew Ng Deeplearning.AI \nTable of Contents \n \n1 Why Machine Learning Strategy  \n2 How to use this book to help your team  \n3 Prerequisites and Notation  \n4 Scale drives machine learning progress  \n5 Your development and test sets  \n6 Your dev and test sets should come from the same distribution  \n7 How large do the dev/test sets need to be?  \n8 Establish a single-number evaluation metric for your team to optimize  \n9 Optimizing and satisficing metrics  \n10 Having a dev set and metric speeds up iterations  \n11 When to change dev/test sets and metrics  \n12 Takeaways: Setting up development and test sets  \n13 Build your first system quickly, then iterate  \n14 Error analysis: Look at dev set examples to evaluate ideas  \n15 Evaluating multiple 

In [29]:
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [31]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [32]:
document_answer_gen

[Document(page_content=' \n \n \n \n \n \n \nMachine L earning Year ning is a   \ndeeplear ning. ai pr oject. \n \n \n \n \n \n \n \n \n \n \n© 2018 Andrew Ng.  All Rights R eserv ed. \n \n  \nPage 2 Machine Learning Yearning-Draft Andrew Ng Deeplearning.AI \nTable of Contents \n \n1 Why Machine Learning Strategy  \n2 How to use this book to help your team  \n3 Prerequisites and Notation  \n4 Scale drives machine learning progress  \n5 Your development and test sets  \n6 Your dev and test sets should come from the same distribution  \n7 How large do the dev/test sets need to be?  \n8 Establish a single-number evaluation metric for your team to optimize  \n9 Optimizing and satisficing metrics  \n10 Having a dev set and metric speeds up iterations  \n11 When to change dev/test sets and metrics  \n12 Takeaways: Setting up development and test sets  \n13 Build your first system quickly, then iterate  \n14 Error analysis: Look at dev set examples to evaluate ideas  \n15 Evaluating multiple 

In [33]:
len(document_answer_gen)

40

In [34]:
from langchain.chat_models import ChatOpenAI

In [35]:
llm_ques_gen_pipeline = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.3
)

In [36]:

prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [37]:
from langchain.prompts import PromptTemplate

In [38]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [39]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [40]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [41]:
from langchain.chains.summarize import load_summarize_chain

In [42]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)